In [13]:
import pandas as pd

filename = 'otodom_1000_luxury_level.csv'
df = pd.read_csv(filename, sep=',')

df = df[df['luxury_level'] >= 0]
df = df[df['price'] >= 0]
df = df[df['area'] >= 0]
df['building_floors'] = df['building_floors'].fillna(0).astype(int)
df.describe()

,price,area,rooms,build_year,location_lat,location_lon,building_floors,luxury_level
count,4.350000e+02,435.000000,435.000000,368.000000,435.000000,435.000000,435.000000,435.000000
mean,9.173760e+05,55.475264,2.528736,1995.432065,50.057426,19.958076,4.620690,6.335632
std,7.669092e+05,25.772035,1.007336,32.755319,0.030718,0.048726,3.189273,1.863423
min,1.790000e+05,14.600000,1.000000,1876.000000,49.977213,19.843990,0.000000,1.000000
25%,6.170000e+05,39.215000,2.000000,1975.750000,50.029306,19.919715,3.000000,5.000000
50%,7.690000e+05,50.650000,2.000000,2008.000000,50.063220,19.955259,4.000000,7.000000
75%,9.967000e+05,63.445000,3.000000,2020.000000,50.081964,19.989550,5.000000,8.000000
max,1.350000e+07,253.000000,6.000000,2027.000000,50.117880,20.140287,40.000000,9.000000


In [ ]:
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
X = df[['area', 'rooms', 'build_year',
       'location', 'location_lat', 'location_lon', 'heating',
       'floor', 'building_floors', 
       'state', 'market', 'ownership', 'ad_type',
       'luxury_level']]
y = df['price']

X = pd.get_dummies(X, columns=['location', 'heating', 'state', 'market', 'ownership', 'ad_type', 'floor'], drop_first=True)

# Rename columns to valid feature names
X.columns = [col.replace('[', '').replace(']', '').replace('<', '').replace('>', '').replace(' ', '_') for col in X.columns]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = xgb.XGBRegressor(
    n_estimators=1000,
    learning_rate=0.1,
    max_depth=6,
    min_child_weight=1,
    subsample=0.8,
    colsample_bytree=0.8,
    gamma=0,
    reg_alpha=0.1,
    reg_lambda=1
)
model.fit(X_train, y_train)

from sklearn.metrics import mean_squared_error, r2_score
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {rmse}')
print(f'R^2 Score: {r2}')

Mean Squared Error: 176179.27531297927
R^2 Score: 0.8108617790162274
